# Request and Response in Workflows


**Executors** are where computation happens. When an executor emits a message, it flows along the edges to downstream executors which activate when their dependent data becomes available. 

Beyond the internal flow, executors can also pause and send requests outside the workflow to wait for responses which is useful for any operations that depend on external systems or humans.

Now that we understand how tool approvals work to control agent actions, let's explore how Agent Framework handles HITL interactions at the workflow level. In this section, we'll examine how executors can pause execution to request human input or approval, wait for responses, and then continue processing based on that feedback.

## Learning Objectives

By completing this section, you will:

- **Understand** the request-response pattern of Agent Framework
- **Build** a complete content generation system with human approval requirements
- **Implement** multi-executor workflows
- **Apply** these concepts by building your own HITL approval system


### Key Steps in Request/Response message handling

Executors have two primary mechanisms for request-response interactions:

1. Sending requests via `ctx.request_info`:

    `ctx.request_info(request_data=request_data, response_type=response_type)`

**Purpose:** Pause workflow execution and request external input

**What happens:**

- Workflow emits a `RequestInfoEvent` containing your request data
- Workflow execution pauses at this point
- The event surfaces to the external application
- External system processes the request and provides a response

2. Handling responses via `@response_handler`:

    ```python
    @response_handler
    async def on_human_feedback(
        self,
        original_request: HumanFeedbackRequest,
        feedback: str,
        ctx: WorkflowContext[AgentExecutorRequest, str],
    ) -> None:
        # Process response and continue
    ```

**Purpose:** Define how to handle responses when they arrive

**What happens:**

- Automatically registered to handle responses matching the request/response types
- Framework routes the response to this method when received
- You can then continue processing or send messages to other executors

Executors can send requests directly using `ctx.request_info()` and handle responses using the `@response_handler` decorator


### Example - Content Generation with HITL

In this example, we'll build a content generation and approval workflow where AI-generated responses must receive human approval before being delivered to users. 

#### Key Components

```
User Query → Worker (AI Generator) → Reviewer (Escalator) → Human Moderator
                ↑                           ↓
                └──── Approval Decision ────┘
                              ↓
                        Final Output → User
```

The system consists of two executors operating in a feedback loop:

1. **Worker** generates AI responses and sends them for review
2. **Reviewer** escalates to a human for approval/rejection
3. **Human** makes the decision (simulated in this demo)
4. **Reviewer** forwards the decision back to Worker
5. **Worker** either emits approved content or regenerates based on feedback

---

### Setup

First, let's validate our environment and create the Azure OpenAI client.


In [2]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

load_dotenv()

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_AI_API_VERSION")
credential = AzureCliCredential()


chat_client = AzureOpenAIChatClient(
    endpoint=project_endpoint,
    ad_token=credential.get_token("https://cognitiveservices.azure.com/.default").token,
    deployment_name=deployment_name,
    api_version=api_version
)


In [5]:
from collections.abc import Mapping
from dataclasses import dataclass
from pathlib import Path
from typing import Any
from uuid import uuid4


from agent_framework import ( 
    ChatMessage,
    Executor,
    AgentRunUpdateEvent,
    AgentRunResponseUpdate,
    Contents,
    FunctionCallContent,
    FunctionResultContent,
    Role,
    WorkflowAgent,
    WorkflowBuilder,
    ChatClientProtocol,
    WorkflowContext,
    handler,
    response_handler,
)

### Implementation

First, let's define the necessary data models

In [6]:
# The data that flows betweeen executors inside the workflows
@dataclass
class ReviewRequest:
    """Structured request passed from Worker to Reviewer for evaluation."""

    request_id: str
    user_messages: list[ChatMessage]
    agent_messages: list[ChatMessage]

# Response data that comes after review
@dataclass
class ReviewResponse:
    """Structured response from Reviewer back to Worker."""

    request_id: str
    feedback: str
    approved: bool

@dataclass
class HumanReviewRequest:
    """A request message type for escalation to a human reviewer."""

    agent_request: ReviewRequest | None = None

Understanding executor logic is crucial

Then we have two executors, `Worker` and `Reviewer`

**`Worker`**

The Worker both performs LLM inference and manages the review lifecycle. It receives user queries, calls the LLM to generate responses, sends them for human review, and handles feedback by either emitting approved responses to users or regenerating improved versions based on reviewer comments.

In [10]:
class Worker(Executor):
    """Executor that generates responses and incorporates feedback when necessary."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client
        self._pending_requests: dict[str, tuple[ReviewRequest, list[ChatMessage]]] = {}

    @handler
    async def handle_user_messages(self, user_messages: list[ChatMessage], ctx: WorkflowContext[ReviewRequest]) -> None:
        print("Worker: Received user messages, generating response...")

        # Initialize chat with system prompt.
        messages = [ChatMessage(role=Role.SYSTEM, text="You are a helpful assistant. Be concise.")]
        messages.extend(user_messages)

        print("Worker: Calling LLM to generate response...")
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: Response generated: {response.messages[-1].text}")

        # Add agent messages to context.
        messages.extend(response.messages)

        # Create review request and send to Reviewer.
        request = ReviewRequest(request_id=str(uuid4()), user_messages=user_messages, agent_messages=response.messages)
        print(f"Worker: Sending response for review (ID: {request.request_id[:8]})")
        await ctx.send_message(request)

        # Track request for possible retry.
        self._pending_requests[request.request_id] = (request, messages)

    @handler
    async def handle_review_response(self, review: ReviewResponse, ctx: WorkflowContext[ReviewRequest]) -> None:
        print(f"Worker: Received review for request {review.request_id[:8]} - Approved: {review.approved}")

        if review.request_id not in self._pending_requests:
            raise ValueError(f"Unknown request ID in review: {review.request_id}")

        request, messages = self._pending_requests.pop(review.request_id)

        if review.approved:
            print("Worker: Response approved. Emitting to external consumer...")
            contents: list[Contents] = []
            for message in request.agent_messages:
                contents.extend(message.contents)

            # Emit approved result to external consumer via AgentRunUpdateEvent.
            await ctx.add_event(
                AgentRunUpdateEvent(self.id, data=AgentRunResponseUpdate(contents=contents, role=Role.ASSISTANT))
            )
            return

        print(f"Worker: Response not approved. Feedback: {review.feedback}")
        print("Worker: Regenerating response with feedback...")

        # Incorporate review feedback.
        messages.append(ChatMessage(role=Role.SYSTEM, text=review.feedback))
        messages.append(
            ChatMessage(role=Role.SYSTEM, text="Please incorporate the feedback and regenerate the response.")
        )
        messages.extend(request.user_messages)

        # Retry with updated prompt.
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: New response generated: {response.messages[-1].text}")

        messages.extend(response.messages)

        # Send updated request for re-review.
        new_request = ReviewRequest(
            request_id=review.request_id, user_messages=request.user_messages, agent_messages=response.messages
        )
        await ctx.send_message(new_request)

        # Track new request for further evaluation.
        self._pending_requests[new_request.request_id] = (new_request, messages)

2. **`ReviewerWithHumanInTheLoop`**
The Reviewer acts as a human escalation point in the review process. It receives response proposals from the Worker, escalates them to a human for evaluation via ctx.request_info(), and forwards the human's decision back to the Worker.


In [11]:
class ReviewerWithHumanInTheLoop(Executor):
    """Executor that always escalates reviews to a human manager."""

    # when we create an instance, the id is needed so we can send responses back to it
    def __init__(self, worker_id: str, reviewer_id: str | None = None) -> None:
        unique_id = reviewer_id or f"{worker_id}-reviewer"  # if a reviewer id is provided we use it and if not we create a custom ID
        super().__init__(id=unique_id)
        self._worker_id = worker_id

    # Handler 1 - receives the initial review request, runs when the Reviewer executor received a ReviewRequest message
    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext) -> None:

        print(f"Reviewer: Evaluating response for request {request.request_id[:8]}...")
        print("Reviewer: Escalating to human manager...")

        # Forward the request to a human manager by sending a HumanReviewRequest.
        await ctx.request_info(request_data=HumanReviewRequest(agent_request=request), response_type=ReviewResponse)

    # Handler 2 - receives the human response and 
    @response_handler
    async def accept_human_review(
        self,
        original_request: HumanReviewRequest,
        response: ReviewResponse,
        ctx: WorkflowContext[ReviewResponse],
    ) -> None:
        # Accept the human review response and forward it back to the Worker.
        print(f"Reviewer: Accepting human review for request {response.request_id[:8]}...")
        print(f"Reviewer: Human feedback: {response.feedback}")
        print(f"Reviewer: Human approved: {response.approved}")
        print("Reviewer: Forwarding human review back to worker...")
        await ctx.send_message(response, target_id=self._worker_id)

Then we can finally build the workflow. Ours has two edges and ...


In [12]:
print("Starting Workflow Agent with Human-in-the-Loop Demo")
print("=" * 50)

print("Building workflow with Worker-Reviewer cycle...")
# Build a workflow with bidirectional communication between Worker and Reviewer,
# and escalation paths for human review.
agent = (
    WorkflowBuilder()
    .register_executor(
        lambda: Worker(
            id="sub-worker",
            chat_client=chat_client,
        ),
        name="worker",
    )
    .register_executor(
        lambda: ReviewerWithHumanInTheLoop(worker_id="sub-worker"),
        name="reviewer",
    )
    .add_edge("worker", "reviewer")  # Worker sends requests to Reviewer
    .add_edge("reviewer", "worker")  # Reviewer sends feedback to Worker
    .set_start_executor("worker")
    .build()
    .as_agent()  # Convert workflow into an agent interface
)

Starting Workflow Agent with Human-in-the-Loop Demo
Building workflow with Worker-Reviewer cycle...


Finally, we can see how this works in action - two runs, context, receiving and sending assembled things with correct types

In [ ]:
print("Running workflow agent with user query...")
print("Query: 'Write code for parallel reading 1 million files on disk and write to a sorted output file.'")
print("-" * 50)


# Run the agent with an initial query.
response = await agent.run(
    "What is the importance of learning programming?"
)

# Locate the human review function call in the response messages.
human_review_function_call: FunctionCallContent | None = None
for message in response.messages:
    for content in message.contents:
        if isinstance(content, FunctionCallContent) and content.name == WorkflowAgent.REQUEST_INFO_FUNCTION_NAME:
            human_review_function_call = content

# Handle the human review if required.
if human_review_function_call:
    print("\n Workflow paused - Human review required")
    
    # Parse the human review request arguments.
    human_request_args = human_review_function_call.arguments
    if isinstance(human_request_args, str):
        request: WorkflowAgent.RequestInfoFunctionArgs = WorkflowAgent.RequestInfoFunctionArgs.from_json(
            human_request_args
        )
    elif isinstance(human_request_args, Mapping):
        request = WorkflowAgent.RequestInfoFunctionArgs.from_dict(dict(human_request_args))
    else:
        raise TypeError("Unexpected argument type for human review function call.")

    request_payload: Any = request.data
    if not isinstance(request_payload, HumanReviewRequest):
        raise ValueError("Human review request payload must be a HumanReviewRequest.")

    agent_request = request_payload.agent_request
    if agent_request is None:
        raise ValueError("Human review request must include agent_request.")

    request_id = agent_request.request_id
    # Mock a human response approval for demonstration purposes.
    human_response = ReviewResponse(request_id=request_id, feedback="Approved", approved=True)

    # Create the function call result object to send back to the agent.
    human_review_function_result = FunctionResultContent(
        call_id=human_review_function_call.call_id,
        result=human_response,
    )
    # Send the human review result back to the agent.
    response = await agent.run(ChatMessage(role=Role.TOOL, contents=[human_review_function_result]))
    print(f"Final Agent Response: {response.messages[-1].text}")

print("=" * 50)
print("Workflow completed!")

Running workflow agent with user query...
Query: 'Write code for parallel reading 1 million files on disk and write to a sorted output file.'
--------------------------------------------------
Worker: Received user messages, generating response...
Worker: Calling LLM to generate response...
Worker: Response generated: Learning programming is important because it:

1. **Enhances problem-solving skills**: Programming teaches logical thinking and structured problem-solving.
2. **Boosts career opportunities**: It opens doors to high-demand and well-paying tech jobs across industries.
3. **Facilitates innovation**: Enables the creation of software, apps, and systems to solve real-world problems.
4. **Improves automation**: Helps automate repetitive tasks, increasing efficiency.
5. **Develops digital literacy**: Strengthens understanding of how technology works.
6. **Promotes creativity**: Encourages building custom solutions and experimenting with ideas.
7. **Prepares for the future**: Esse

### Exercise - Create Your Own Workflow with HITL

## Summary & Recap

In this section, you've learned how to create interactive workflows that can pause execution to request input from external sources (like humans or other systems) and then resume once a response is provided.

### Key Concepts

### What You Built

### Key Patterns